# Libreria AutomLib.

<img src = "./imagenes/autom.png" width = "350px">

---


- **Autor:** Juan Esteban Cepeda Baena$^{1}$.
- **Code License:** MIT
- **Email:** juancepeda.gestion@gmail.com / jecepedab@unal.edu.co
- **Google Site:** https://sites.google.com/view/juancepeda/
- **Linkedin:** https://www.linkedin.com/in/juan-e-cepeda-gestion/

---

$^{1}$ Estudiante de Ciencias de la Computación y Administración de Empresas de la Universidad Nacional de Colombia.

In [1]:
# Import libraries.
import numbers
import math
import numpy as np
import string
import re
import matplotlib.pyplot as plt
import copy

### 1. Clase Procesador Archivos.

---

Esta clase permite crear objetos para procesar la información de los archivos que codifican las características de un autómata. En particular, se encarga de determinar el alfabeto, estados, estado inicial, estados de aceptación, y función delta de transiciones entre estados, a partir de la ruta en donde está almacenado el archivo. Finalmente, retorna un objeto de la forma $M = (\Sigma, Q, q_0, F, \delta)$.

In [2]:
class ProcesadorArchivos:
    
    def __init__(self, filename = ""):
        
        # Obtiene la información del archivo.
        try:
            file_info = self.obtenerInformacionArchivo(filename)
            #print("file info: ", file_info)

            # Obtiene palabras clave del archivo.
            keywords = "#alphabet #states #initial #accepting #transitions".split()
            keywords_index = [file_info.index(keywords[i]) for i in range(len(keywords))]

            # Obtiene parámetros del autómata.
            automata_name = file_info[0]
            self.alphabet = self.procesarCaracteresAutomata(file_info[(keywords_index[0] + 1) : (keywords_index[1])]) # Obtener alfabeto.
            self.states = file_info[(keywords_index[1] + 1) : (keywords_index[2])] # Obtener estados.
            self.initial_state = file_info[(keywords_index[2] + 1) : (keywords_index[3])][0]
            self.valid_states = file_info[(keywords_index[3] + 1) : (keywords_index[4])]
            self.delta =  self.procesarTransicionesAutomata(file_info[(keywords_index[4] + 1):])
            self.automata_type = {"#!dfa": 0, "#!nfa": 1, "#!nfae": 2}[automata_name]
            if self.automata_type == 2: 
                if "$" not in self.alphabet: 
                    self.alphabet.append("$")
        except: 
            print("Se ha producido un error al intentar leer el archivo. Por favor verifique que la estructura del archivo es correcta.")
                
    """
    Procesa los caracteres ingresados por el usuario en el archivo.
    """
    def procesarCaracteresAutomata(self, lista_caracteres):
        """
        comando: "a-c"
        """
        caracteres = self.obtenerCaracteres()
        secuencia_final = list()
        for comando in lista_caracteres: 
            if len(comando) > 1:
                primer_caracter = comando[0]
                ultimo_caracter = comando[2]
                secuencia_caracteres = caracteres[caracteres.index(primer_caracter):caracteres.index(ultimo_caracter)+1]
                for letra in secuencia_caracteres:
                    secuencia_final.append(letra)
            else: secuencia_final.append(comando)
        return secuencia_final
    
    """
    Construya la función delta de transiciones del autómata, con base en la información
    del archivo proporcionada por el usuario.
    """
    def procesarTransicionesAutomata(self, lista_transiciones):
        """
        Entrada: ['s0:b>s1', 's0:$>s0,s1', 's1:a>s0', 's1:b>s2,s3', 's1:$>s2,s3', 's2:a>s2,s3', 
        's2:$>s2s', 's3:c>s1', 's3:$>s1,s2']
        Salida:
        delta = [
              ["Q0", "Q0", "a"], 
              ["Q0", "Q1", "a"], 
              ["Q1", "Q1", "b"], 
              ["Q1", "Q0", "a"], 
              ["Q1", "Q2", "a"]
            ]
        """
        delta = list()
        for transicion in lista_transiciones: 
            temp = transicion.split(":")

            # Estado de salida.
            estado_salida = temp[0]

            # Obtener instrucción.
            temp2 = temp[1].split(">")
            instruccion = temp2[0]

            # Lista de estados de llegada.
            estados_de_llegada = temp2[1].split(";")
            for estado_llegada in estados_de_llegada:
                delta.append([estado_salida, estado_llegada, instruccion])
        return delta
    
    """
    Construye una lista de todos los caractéres posibles que el usuario puede digitar.
    """
    def obtenerCaracteres(self):
        caracteres = list()
        for i in range(len(string.ascii_lowercase)):
            caracteres.append(string.ascii_lowercase[i])
        for i in range(len(string.ascii_lowercase)):
            caracteres.append(string.ascii_lowercase.upper()[i])
        caracteres.insert(14, "ñ")
        numeros = "0 1 2 3 4 5 6 7 8 9".split()
        for i in range(len(numeros)):
            caracteres.append(numeros[i])
        return caracteres
    
    """
    Carga el archivo y lo convierte en una lista.
    """
    def obtenerInformacionArchivo(self, filename):

        file = open(filename, "r")
        file_string = file.read().split("\n")
        file_info = list()
        file.close()
        for i in range(len(file_string)):
            if file_string[i] != "":
                file_info.append(file_string[i])
        return file_info    

### 2. Clase Procesador Cadenas.

---

Esta clase es la encargada de formatear y calcular características de los distintos procesamientos de una cadena por parte de un autómata. Algunas de sus funciones son: verificar si un procesamiento es de aceptación o no, verificar si al menos un procesamiento terminó en un estado de aceptación, formatear la información del procesamiento de una cadena para luego imprimir por pantalla la secuencia de estados e instrucciones, guardar información de los procesamientos en distintos archivos, obtener información de los procesamientos (número de trayectorias totales, de aceptación, rechazo o abortadas, etc), obtener los procesamientos más cortos, entre otras funcionalidades.

In [3]:
class ProcesamientoCadenas:
    
    def __init__(self, states):
        self.states = states

    def corroborarAceptacion(self, acceptanceList):
        if 1 in acceptanceList: return 1
        else: return 0   
    
    """
    Recorrido: 00010010022001010 => States index
    """
    def obtenerSecuenciaEstadoInstruccion(self, cadena, recorrido):
        secuencia = list()    
        for i in range(len(recorrido)): 
            state_name = self.states[int(recorrido[i])]
            try:
                if cadena[i] == "$":
                    secuencia.append([state_name, cadena[i+1:].replace("$", "")])
                else:
                    secuencia.append([state_name, cadena[i:].replace("$", "")])
            except:
                secuencia.append([state_name, ""])
        return secuencia 
    
    def validChain(self, trayectorias):
        for trayectoria in trayectorias: 
            if trayectoria[1] == 1:
                return True
        return False
    
    def clasificarTrayectorias(self, trayectorias):
        
        # Get info from trayectories.
        accepted_trayectories = list()
        rejected_trayectories = list()
        aborted_trayectories = list()
        
        # Classify by trayectory type.
        for trayectoria in trayectorias: 
            if trayectoria[1] == 1:
                accepted_trayectories.append(trayectoria)
            elif trayectoria[1] == 0:
                aborted_trayectories.append(trayectoria)
            elif trayectoria[1] == -1:
                rejected_trayectories.append(trayectoria)
        return [accepted_trayectories, rejected_trayectories, aborted_trayectories]
    
    def getTrayectoriesInfo(self, trayectorias):
        
        # Accepted.
        accepted = self.validChain(trayectorias)
        if accepted: accepted = "Si"
        else: accepted = "No"
        
        # Get info from trayectories.
        clasificador = self.clasificarTrayectorias(trayectorias)
        accepted_trayectories = clasificador[0]
        rejected_trayectories = clasificador[1]
        aborted_trayectories = clasificador[2]
                        
        # Get shortest trayectory for each type.
        shortest_accepted = self.getShortestTrayectory(accepted_trayectories)
        shortest_aborted = self.getShortestTrayectory(aborted_trayectories)
        shortest_rejected = self.getShortestTrayectory(rejected_trayectories)
        
        # Compute trayectories features.
        
        # número de posibles procesamientos.
        total_trayectories = len(trayectorias) 
        total_trayectories_accepted = len(accepted_trayectories)
        total_trayectories_rejected = len(rejected_trayectories)
        total_trayectories_aborted = len(aborted_trayectories)
    
        return[shortest_accepted,  
               shortest_rejected, 
               shortest_aborted,
               total_trayectories, 
               total_trayectories_accepted,
               total_trayectories_aborted,
               total_trayectories_rejected,
               accepted
              ]
    
    """
    Input Example: [0110, 1, ab]
    Output Example: [0110, aab]
    """
    def getShortestTrayectory(self, trayectorias):
        
        if len(trayectorias) > 0:
            
            #print("trayectorias es mayor a 0!!!")
            
            recorridos = list()
            for trayectoria in trayectorias: 
                recorrido = trayectoria[0]
                recorridos.append(recorrido)

            shortest_path_length = len(recorridos[0])
            shortest_path_index = 0

            for recorrido in recorridos: 
                if len(recorrido) < shortest_path_length: 
                    shortest_path_length = len(recorrido)
                    shortest_path_index = recorridos.index(recorrido)
            return trayectorias[shortest_path_index]
        return ["", "", ""]
    
    """
    Entrada: [['Q0', 'aba'], ['Q1', 'ba'], ['Q1', 'a'], ['Q0', '']], resultado = 1, -1, 0
    Output: ['Q0', 'aba'] -> ['Q1', 'ba'] -> ['Q1', 'a'] -> ['Q0', ''] -> Aceptado
    """
    def imprimirSecuencias(self, secuencias, resultado):
        for secuencia in secuencias: 
            if len(secuencias) > 1: print("Procesamiento No.", secuencias.index(secuencia) + 1)
            for estado in secuencia: 
                if secuencia.index(estado) == len(secuencia) - 1:
                    print(estado, "->", resultado, end = "\n")
                else: 
                    print(estado, "->", end = " ")
            print("")
    
    def guardarSecuencias(self, fileObject, secuencias, cadena, resultado):
        fileObject.write(cadena + "\n")
        for secuencia in secuencias: 
            for transicion in secuencia: 
                fileObject.write(str(transicion) + " ->" + "\t")
            fileObject.write(resultado + "\n")

    # Obtener secuencias de los trayectos o procesamientos.
    def obtenerTrayectos(self, procesamientos):
        
        secuencia_trayectos = list()
        for procesamiento in procesamientos: 
            memoria_instrucciones = procesamiento[2]
            memoria_recorrido = procesamiento[0]
            secuencia = self.obtenerSecuenciaEstadoInstruccion(
                memoria_instrucciones,
                memoria_recorrido
            )
            secuencia_trayectos.append(secuencia)
        return secuencia_trayectos     
    
    """
    La memoria del trayecto de los procesamientos abortados, sólo retorna la parte de 
    la cadena que logró procesar, incluyendo las lambda transiciones en el caso de los AFNLambda;
    el comportamiento esperado, es que también retorno la parte de la cadena que no se logró 
    procesar, y que terminó abortado el procedimiento.
    
    Entrada: 
        Cadena:  abbaab
        Trayectorias:  [['00110012', 1, 'abbaab$'], ['00112', 0, 'abb$'], ['00122', 0, 'ab$b']]
    Salida: 
        Trayectorias:  [['00110012', 1, 'abbaab$'], ['00112', 0, 'abb$aab'], ['00122', 0, 'ab$baab']]
    """
    def arreglarTrayectoriaDeAbortados(self, trayectorias, cadena):
        
        # Check all the trayectories.
        for trayectoria in trayectorias: 
            # Get the aborted trayectories.
            if trayectoria[1] == 0:
                cadena_procesada = trayectoria[2].replace("$", "")
                cadena_no_procesada = cadena.replace(cadena_procesada, "", 1)
                trayectoria[2] = trayectoria[2] + cadena_no_procesada
        return trayectorias

### 3. Clase Alfabeto.

In [4]:
class Alfabeto: 
    
    def __init__(self, simbolos):
        self.simbolos = simbolos
        self.simbolos.sort()
    
    def generarCadenaAleatoria(self, n): 
        cadena = ""
        caracteres = self.simbolos.copy()
        if "$" in caracteres: 
            caracteres.remove("$")
        for i in range(n):
            random_number = np.random.randint(len(caracteres))
            cadena += self.simbolos[random_number]
        return cadena

### 4. Clase Autómata

---

La clase autómata incorpora la estructura y funcionalidad básica de los autómatas a partir de un conjunto de parámetros dados por el usuario (alfabeto, estado inicial, estados, estados de aceptación, función delta), o a partir de la ruta de un archivo.

<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/DFAexample.svg/400px-DFAexample.svg.png" width = "200px">

In [13]:
class Automata:
    
    def __init__(self, alphabet = [], initial_state = "Q0", states = ["Q0"], valid_states = [], delta = [], filename = ""):
                
        # Construir autómata con los parámetros tradicionales.
        if filename == "":
            self.delta = delta
            self.alphabet = Alfabeto(alphabet)
            self.states = states
            self.initial_state = states.index(initial_state)
            self.valid_states = self.getValidStatesIndex(states, valid_states)
            self.adjMatrix = self.generarMatrizAdyacencia(states, delta)
    
        # Construir autómata utilizando archivo.
        else: 
            
            lector_archivos = ProcesadorArchivos(filename) 
            
            self.delta = lector_archivos.delta
            self.alphabet = Alfabeto(lector_archivos.alphabet.copy())
            self.states = lector_archivos.states.copy()
            self.initial_state = self.states.index(lector_archivos.initial_state)
            self.valid_states = self.getValidStatesIndex(lector_archivos.states, lector_archivos.valid_states)
            self.adjMatrix = self.generarMatrizAdyacencia(lector_archivos.states, lector_archivos.delta)
           
            self.automata_type = lector_archivos.automata_type
            print("")
            
        self.output_word = {1: "Aceptado", -1: "Rechazado", 0: "Abortado"}
        
        # Inicializar procesador de cadenas.
        self.procesadorCadenas = ProcesamientoCadenas(self.states)

    def getValidStatesIndex(self,  states, valid_states):
        validStates = list()
        for state in valid_states:
            validStates.append(states.index(state))
        return validStates
    
    def generarMatrizAdyacencia(self, states, delta):
        
        # Create matrix object.
        adjMatrix = list()
        
        # Number of layers of the adyacency matrix (depth).
        n = len(states)
        for i in range(len(self.alphabet.simbolos)):
              adjMatrix.append(np.zeros((n, n)))
                
        # Delta codifies the edges between nodes.
        for edge in delta:
            
            # Example: edge = ["q0", "q1", "a"]
            # Get states number.
            first_state = states.index(edge[0])
            second_state = states.index(edge[1])
            
            # Get instruction.
            instruction = self.alphabet.simbolos.index(edge[2])
            
            # Add new edge.
            adjMatrix[instruction][first_state][second_state] = 1
            
        # Return adyacency matrix.
        return adjMatrix

    # Utility functions.
    def obtenerNumerosEstadosAccesibles(self, current_state, instruction):
        layer = self.adjMatrix[instruction]
        bridges = layer[current_state]
        numbers = list()
        for b in range(len(bridges)):
            if bridges[b] == 1: numbers.append(b)
        return numbers
    
    """
    Encuentra todos los estados limbo del autómata.
    """
    def hallarEstadosLimbo(self):  
        lista_busqueda = list()
        for state in self.states:
            # Es diferente del estado inicial (el estado inicial nunca es limbo).
            if state != self.states[self.initial_state]:
                valid = False
                # No pertenece al conjunto de estados de aceptación.
                for valid_state in self.valid_states:
                    if state == self.states[valid_state]:
                        valid = True
                        break
                if not valid:
                    lista_busqueda.append(state)
                    
        estados_limbo = list()
        for estado_busqueda in lista_busqueda:
            
            estados_explorados = list()
            estados_explorados.append(estado_busqueda)
            if self.esEstadoLimbo(estado_busqueda, estados_explorados):
                estados_limbo.append(estado_busqueda)
                
        return estados_limbo
    
    """
    Determina si un estado es o no limbo.
    Entrada: NombreEstado.
    Salida: True/False.
    """
    def esEstadoLimbo(self, estado, estados_explorados):
        
        # Si el estado pertenece a los estados de aceptación, retorne False.
        if estado in self.getValidStates():
            return False

        # Busque las transiciones del estado actual siempre que el estado actual no sea de validación.
        for transicion in self.delta: 
            if transicion[0] == estado: 
                nuevo_estado = transicion[1]
                if nuevo_estado not in estados_explorados:
                    estados_explorados.append(nuevo_estado)
                    esEstadoLimbo = self.esEstadoLimbo(nuevo_estado, estados_explorados)
                    if not esEstadoLimbo:
                        return False           
        return True
    
    """
    Hallar Estados Inaccesibles.
    """
    def hallarEstadosInaccesibles(self):
                
        # Inicializar variables.
        estados_alcanzables = list()
        estado_inicial = self.getInitialState()
        estados_alcanzables.append(estado_inicial)
        
        self.obtenerEstadosAlcanzablesDesdeEstado(estado_inicial, estados_alcanzables)       
        estados_inaccesibles = list()
        for state in self.states: 
            if state not in estados_alcanzables: 
                estados_inaccesibles.append(state)
        return estados_inaccesibles
    
    """
    Actualiza lista de estados_alcanzables recursivamente.
    """
    def obtenerEstadosAlcanzablesDesdeEstado(self, estado, estados_alcanzables):   
        
        # Obtener estados alcanzables desde el estado de entrada.
        for transicion in self.delta: 
            if transicion[0] == estado: 
                nuevo_estado = transicion[1]
                if nuevo_estado not in estados_alcanzables:    
                    estados_alcanzables.append(nuevo_estado)
                    self.obtenerEstadosAlcanzablesDesdeEstado(nuevo_estado, estados_alcanzables)

    """
    Método para imprimir donde se vean los estados, 
    estado inicial, estados de aceptación, estados inaccesibles, estados limbo y tabla de 
    transiciones en formato de entrada.
    """
    def toString(self): 
        
        # Print automata type.
        name_type = {0: "#!dfa", 1: "#!nfa", 2: "#!nfae"}[self.automata_type]
        print(name_type)
        
        print("#states")
        for state in self.states: 
            print(state)
            
        print("#initial")
        print(self.getInitialState())
        
        print("#accepting")
        for valid in self.getValidStates(): 
            print(valid)
        
        print("#inaccessible")
        for inacc in self.hallarEstadosInaccesibles(): 
            print(inacc)
        
        print("#limbo")
        for limbo in self.hallarEstadosLimbo(): 
            print(limbo)
        
        print("#transitions")
        for state in self.states: 
            for instruction in self.alphabet.simbolos: 
                estados_transicion = list()
                for transicion in self.delta: 
                    if transicion[0] == state and transicion[2] == instruction: 
                        estados_transicion.append(transicion[1])
                if len(estados_transicion) > 0:
                    print(state + ":" + instruction + ">", end = "")
                    transition = ""
                    for e in estados_transicion: 
                        transition += e 
                        if estados_transicion.index(e) + 1 != len(estados_transicion): 
                            transition += ";"
                    print(transition)
                
    """
    Método para imprimir donde se vean los estados, 
    estado inicial, estados de aceptación, estados inaccesibles, estados limbo y tabla de 
    transiciones en formato usual.
    """
    def toString2(self): 
        
        print("Estados: ")
        print(self.states)
        print("")
        
        print("Estado Inicial: ")
        print(self.getInitialState())
        print("")
        
        print("Estados de Aceptación: ")
        print(self.getValidStates())
        print("")
        
        print("Estados Inaccesibles: ")
        print(self.hallarEstadosInaccesibles())
        print("")
        
        print("Estados Limbo: ")
        print(self.hallarEstadosLimbo())
        print("")
        
        print("Tabla de transiciones:")
        for transicion in self.delta: 
            print(transicion)
        print("")
                    
    # Esta función retorna el nombre del estado inicial.
    def getInitialState(self):
        return self.states[self.initial_state]
    
    # Esta función retorna una lista de nombre de los estados de aceptación.
    def getValidStates(self):
        return [self.states[i] for i in self.valid_states]

### 4.1. Autómatas Deterministas.

In [6]:
class AFD(Automata):
    
    def __init__(self, alphabet = [], initial_state = "Q0", states = ["Q0"], valid_states = [], delta = [], filename = ""): 
        super().__init__(alphabet, initial_state, states, valid_states, delta, filename)
        if not self.__verificarAutomataDeterminista():
            raise Exception("Los argumentos ingresados no corresponden a un autómata determinista; No todas las transiciones están definidas")
        self.automata_type = 0 # Determinista.
        
    def procesarCadena(self, cadena):
        return self.output_word[self.procesar(cadena)[1]] == "Aceptado"
    
    def procesarCadenaConDetalles(self, cadena): 
        process = self.procesar(cadena)
        secuencia = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(cadena, process[0])
        self.procesadorCadenas.imprimirSecuencias([secuencia], self.output_word[process[1]])
        return self.output_word[process[1]] == "Aceptado"
           
    def procesarListaCadenas(self, listaCadenas, nombreArchivo, imprimirPantalla): 
        
        try:
            file = open(nombreArchivo, "w+")
        except: 
            print("Error: nombre de archivo inválido.")
            print("Los resultados se guardarán por defecto en ./resultadosAFD.txt")
            file = open("resultadosAFD.txt", "w+")
            
        for cadena in listaCadenas: 
            
            process = self.procesar(cadena)
            secuencia = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(
                cadena,    # cadena
                process[0] # recorrido
            )
            
            file.write(cadena + "\t")
            
            if imprimirPantalla: print(cadena, end = "\n" )
                
            for pareja in secuencia:
                if imprimirPantalla: print(str(pareja) + " ->", end = "\t")
                file.write(str(pareja) + " ->" + "\t")
                
            if process[1] == 1: aceptado = "Aceptado"
            else: aceptado = "Rechazado"
            file.write(aceptado + "\n")
            
            if imprimirPantalla: 
                print("")
                print(aceptado, end = "\n")
                print("")
        
        file.close()
        return "Archivo creado con éxito"

    def procesar(self, cadena, current_state = 0):
        
        # If chain is not null.
        if cadena != "":
            
            # Get next state number given by instruction in the chain.
            newState = self.obtenerNumerosEstadosAccesibles(
                current_state, 
                self.alphabet.simbolos.index(cadena[0])   
            )
            #print("New State: ", newState)
            newState = newState[0]

            # Move to next state.
            memory = self.procesar(cadena[1:], current_state = newState)
            memory[0] = str(current_state) + memory[0]
            return memory

        # If chain is null.
        else: 
            if current_state in self.valid_states:
                return [str(current_state), 1]    # Accepted.
            else:
                return [str(current_state), -1]   # Rejected.
        
    def __verificarAutomataDeterminista(self): 
        # Determina si el autómata es determinista.
        deterministic = True
        for instruction in self.alphabet.simbolos: 
            layer = self.adjMatrix[self.alphabet.simbolos.index(instruction)]
            for row_index in range(len(layer)):
                lista = self.obtenerNumerosEstadosAccesibles(
                    row_index, 
                    self.alphabet.simbolos.index(instruction)
                )
                if len(lista) == 0:
                    deterministic = False
        return deterministic

### 4.2. Autómatas No-Deterministas.

In [7]:
class AFN(Automata):
    def __init__(self, alphabet = [], initial_state = "Q0", states = ["Q0"], valid_states = [], delta = [], filename = ""): 
        super().__init__(alphabet, initial_state, states, valid_states, delta, filename)
        self.automata_type = 1 # No-Determinista.
        
    def procesarCadena(self, cadena):
        trayectorias = self.procesar(cadena)
        if self.procesadorCadenas.validChain(trayectorias): return True
        else: return False
    
    def procesarCadenaConDetalles(self, cadena): 
        trayectorias = self.procesar(cadena)
        print("Procesamientos que terminan en un estado de aceptación: ")
        print("")
        for trayectoria in trayectorias:
            if trayectoria[1] == 1:
                print("Procesamiento No. ", trayectorias.index(trayectoria))
                secuencias = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(
                    cadena, 
                    trayectoria[0]
                )
                self.procesadorCadenas.imprimirSecuencias([secuencias], "Aceptado")
                print(" ")
        if self.procesadorCadenas.validChain(trayectorias): return True
        else: return False
                
    def computarTodosLosProcesamientos(self, cadena):
        
        trayectorias = self.procesar(cadena)
        trayectorias = self.procesadorCadenas.arreglarTrayectoriaDeAbortados(trayectorias, cadena)
        clasificador = self.procesadorCadenas.clasificarTrayectorias(trayectorias)
        
        print("---Procesamientos Aceptados---")
        procesamientos_aceptados = clasificador[0]
        aceptados = self.procesadorCadenas.obtenerTrayectos(procesamientos_aceptados)
        self.procesadorCadenas.imprimirSecuencias(aceptados, "Aceptado")
        file1 = open("./resultadosAFN/procesamientosAceptados.txt", "w+")
        self.procesadorCadenas.guardarSecuencias(file1, aceptados, cadena, "Aceptado")
        file1.close()
        print("")
        
        print("---Procesamientos Rechazados---")
        procesamientos_rechazados = clasificador[1]
        rechazados = self.procesadorCadenas.obtenerTrayectos(procesamientos_rechazados)
        self.procesadorCadenas.imprimirSecuencias(rechazados, "Rechazado")
        file2 = open("./resultadosAFN/procesamientosRechazados.txt", "w+")
        self.procesadorCadenas.guardarSecuencias(file2, rechazados, cadena, "Rechazado")
        file2.close()
        print("")
        
        print("---Procesamientos Abortados---")
        procesamientos_abortados = clasificador[2]
        abortados = self.procesadorCadenas.obtenerTrayectos(procesamientos_abortados)
        self.procesadorCadenas.imprimirSecuencias(abortados, "Abortado")
        file3 = open("./resultadosAFN/procesamientosAbortados.txt", "w+")
        self.procesadorCadenas.guardarSecuencias(file3, abortados, cadena, "Abortado")
        file3.close()
        
        # Retorna el número de procesamientos realizados.
        return len(trayectorias)
    
    def procesarListaCadenas(self, listaCadenas, nombreArchivo, imprimirPantalla): 
        
        try:
            file = open(nombreArchivo, "w+")
        except: 
            print("Error: nombre de archivo inválido.")
            print("Los resultados se guardarán por defecto en ./resultadosAFN.txt")            
            file = open("resultadosAFN.txt", "w+")
            
        for cadena in listaCadenas: 
            
            if imprimirPantalla: 
                print("------Nueva cadena-----")
                print("")
                print(cadena, end = "\n")
            
            # Process chain.
            trayectorias = self.procesar(cadena)
            trayectorias = self.procesadorCadenas.arreglarTrayectoriaDeAbortados(trayectorias, cadena)
            
            # Get info of trayectories.
            info = self.procesadorCadenas.getTrayectoriesInfo(trayectorias)
                    
            # Inicializar lista.
            secuencia = []
            resultado = ""
            
            # If chain was accepted.
            trayectoria_de_aceptacion = info[0][0]
            if trayectoria_de_aceptacion != "":
                secuencia = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(
                    cadena,
                    trayectoria_de_aceptacion
                )
                resultado = "Aceptado"
            else:
                # If chain was rejected.
                trayectoria_de_rechazo = info[1][0]
                if trayectoria_de_rechazo != "":
                    secuencia = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(
                        cadena,
                        trayectoria_de_rechazo
                    )       
                    resultado = "Rechazado"
                else:
                    # If chain was aborted.
                    trayectoria_de_abortado = info[2][0]
                    secuencia = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(
                        cadena, 
                        trayectoria_de_abortado
                    )
                    resultado = "Abortado"

            # Guardar y/o imprimir información de la secuencia.
            if imprimirPantalla:
                self.procesadorCadenas.imprimirSecuencias([secuencia], resultado)
            self.procesadorCadenas.guardarSecuencias(file, [secuencia], cadena, resultado)

            # Más información del procesamiento ejecutado.
            if imprimirPantalla: 
                for element in info[3:]:
                    print(element, end = "\t")
                print("")
                print("")
            file.write(str(info[3]) + "\t") # Numero de posibles procesamientos.
            file.write(str(info[4]) + "\t") # Numero de procesamiento de aceptación.
            file.write(str(info[5]) + "\t") # Número de procesamientos abortados.
            file.write(str(info[6]) + "\t") # Número de procesamientos recahazados
            file.write(str(info[7]) + "\n") # Si o no
            
        file.close()
        return "Archivo creado con éxito"
    
    def procesarCadenaConversion(self, cadena): 
        afd_equivalente = TransformadorAutomatas().AFNtoAFD(self, imprimirResultados = False)
        return afd_equivalente.procesarCadena(cadena)
        
    def procesarCadenaConDetallesConversion(self, cadena):
        afd_equivalente = TransformadorAutomatas().AFNtoAFD(self)
        return afd_equivalente.procesarCadenaConDetalles(cadena, imprimirResultados = False)
    
    def procesarListaCadenasConversion(self, listaCadenas, nombreArchivo, imprimirPantalla):
        afd_equivalente = TransformadorAutomatas().AFNtoAFD(self)
        afd_equivalente.procesarListaCadenas(listaCadenas, nombreArchivo, imprimirPantalla) 
    
    """
    -procesar.
    Output example: 
    [['00000', 0, "aaba"],
     ['000011', 1, "bba"],
     ['00010', 0, "baab"]]
    """
    def procesar(self, cadena, current_state = 0, ultima_instruccion = ""):
              
        #print("Current State: ", current_state)
        #print("Cadena: ", cadena)
        #print("Cadena[1:]", cadena[1:])
        
        # If chain is not empty.
        if cadena != "":
            
            # Get next states numbers given by instruction in the chain.
            newStatesNumbers = self.obtenerNumerosEstadosAccesibles(
                current_state, 
                self.alphabet.simbolos.index(cadena[0])   
            )
                    
            # If there exist new states where to move.
            if newStatesNumbers != []:
                
                acceptance_list = list()
                for newState in newStatesNumbers:
                    result = self.procesar(cadena[1:], newState, cadena[0])
                    for pair in result: 
                        acceptance_list.append(pair)
                for pair in acceptance_list: 
                    
                    # Actualizar la memoria.
                    memory = pair[0]                    
                    pair[0] = str(current_state) + memory
                    
                    # Actualizar la lista de instrucciones que ejecutó
                    # el automata.
                    instructions = pair[2]
                    pair[2] = str(ultima_instruccion) + instructions
                    
                return acceptance_list
 
            # Aborted! Since there is no where to move.
            else: 
                return [[str(current_state), 0, ultima_instruccion]]    # Aborted.       
        
        # If chain is empty.
        else: 
            if current_state in self.valid_states:
                return [[str(current_state), 1, ultima_instruccion]]    # Accepted.
            else:
                return [[str(current_state), -1, ultima_instruccion]]  # Rejected.

### 4.3. Autómatas con transiciones lambda.

In [8]:
class AFNLambda(Automata):
    def __init__(self, alphabet = [], initial_state = "Q0", states = ["Q0"], valid_states = [], delta = [], filename = ""): 
        super().__init__(alphabet, initial_state, states, valid_states, delta, filename)
        if "$" not in self.alphabet.simbolos:  
            raise Exception("Los argumentos ingresados no corresponden a un autómata no determinista con transiciones lambda; $ no está en el alfabeto.")
        self.automata_type = 2 # No-Determinista con Transiciones Lambda.
        
    def calcularLambdaClausura(self, current_state = "q0", index = False):
        
        current_state = self.states.index(current_state)
        
        #[['0123', -1, '$$$'], ['0124', -1, '$$$']]
        
        # Get next states numbers given by lambda.
        lambdaClousure = list()
        lambdaClosureStatesNumbers = self.procesar("", 
                                                   current_state = current_state, 
                                                   lambdaClausura = True)
        for path in lambdaClosureStatesNumbers: 
            trayectory = path[0]
            for state_num in trayectory: 
                if index == False:
                    state_name = self.states[int(state_num)]
                    if state_name not in lambdaClousure:
                        lambdaClousure.append(state_name)
                else: 
                    lambdaClousure.append(int(state_num))
        return lambdaClousure
    
    def calcularLambdaClausuras(self, lista_estados, index = False):
        clausuras = list()
        for current_state in lista_estados: 
            clausuras.append(self.calcularLambdaClausura(current_state, index = index))
        return clausuras
    
    def procesarCadena(self, cadena):
        trayectorias = self.procesar(cadena)
        if self.procesadorCadenas.validChain(trayectorias): return True
        else: return False
    
    def procesarCadenaConDetalles(self, cadena): 
        trayectorias = self.procesar(cadena)
        print("Procesamientos que terminan en un estado de aceptación: ")
        print("")
        for trayectoria in trayectorias:
            if trayectoria[1] == 1:
                print("Procesamiento No. ", trayectorias.index(trayectoria))
                memoria_instrucciones = trayectoria[2]
                memoria_recorrido = trayectoria[0]
                secuencias = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(
                    memoria_instrucciones, 
                    memoria_recorrido
                )
                self.procesadorCadenas.imprimirSecuencias([secuencias], "Aceptado")
                print(" ")
        if self.procesadorCadenas.validChain(trayectorias): return True
        else: return False        

    def computarTodosLosProcesamientos(self, cadena): 
        trayectorias = self.procesar(cadena)
        trayectorias = self.procesadorCadenas.arreglarTrayectoriaDeAbortados(trayectorias, cadena)
        clasificador = self.procesadorCadenas.clasificarTrayectorias(trayectorias)
        
        print("---Procesamientos Aceptados---")
        procesamientos_aceptados = clasificador[0]
        aceptados = self.procesadorCadenas.obtenerTrayectos(procesamientos_aceptados)
        self.procesadorCadenas.imprimirSecuencias(aceptados, "Aceptado")
        file1 = open("./resultadosAFNLambda/procesamientosAceptados.txt", "w+")
        self.procesadorCadenas.guardarSecuencias(file1, aceptados, cadena, "Aceptado")
        file1.close()
        
        print("---Procesamientos Rechazados---")
        procesamientos_rechazados = clasificador[1]
        rechazados = self.procesadorCadenas.obtenerTrayectos(procesamientos_rechazados)
        self.procesadorCadenas.imprimirSecuencias(rechazados, "Rechazado")
        file2 = open("./resultadosAFNLambda/procesamientosRechazados.txt", "w+")
        self.procesadorCadenas.guardarSecuencias(file2, rechazados, cadena, "Rechazado")
        file2.close()
        
        print("---Procesamiento Abortados---")
        procesamientos_abortados = clasificador[2]
        abortados = self.procesadorCadenas.obtenerTrayectos(procesamientos_abortados)
        self.procesadorCadenas.imprimirSecuencias(abortados, "Abortado")
        file3 = open("./resultadosAFNLambda/procesamientosAbortados.txt", "w+")
        self.procesadorCadenas.guardarSecuencias(file3, abortados, cadena, "Abortado")
        file3.close()
        
        # Retorna el número de procesamientos realizados.
        return len(trayectorias)
          
    def procesarListaCadenas(self, listaCadenas, nombreArchivo, imprimirPantalla): 
        try:
            file = open(nombreArchivo, "w+")
        except: 
            print("Error: nombre de archivo inválido.")
            print("Los resultados se guardarán por defecto en ./resultadosAFDLambda.txt")
            file = open("resultadosAFNLambda.txt", "w+")
        
        for cadena in listaCadenas: 
            
            if imprimirPantalla: 
                print("------Nueva cadena-----")
                print("")
                print(cadena, end = "\t")
            
            # Save chain on document.
            file.write(cadena + "\t")
            
            # Process chain.
            trayectorias = self.procesar(cadena)
            trayectorias = self.procesadorCadenas.arreglarTrayectoriaDeAbortados(trayectorias, cadena)
            #print("Trayectorias: ", trayectorias)
            
            # Get info of trayectories.
            #print("Trayectorias: ")
            #print(trayectorias)
            info = self.procesadorCadenas.getTrayectoriesInfo(trayectorias)
            
            secuencia = []
            resultado = ""
            
            # If chain was accepted.
            trayectoria_de_aceptacion = info[0][0]
            
            if trayectoria_de_aceptacion != "":
                
                memoria_instrucciones = info[0][2]
                secuencia = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(
                    memoria_instrucciones,
                    trayectoria_de_aceptacion
                )
                resultado = "Aceptado"
            else:
                # If chain was rejected.
                trayectoria_de_rechazo = info[1][0]
                if trayectoria_de_rechazo != "":
                    
                    memoria_instrucciones = info[1][2]
                    secuencia = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(
                        memoria_instrucciones,
                        trayectoria_de_rechazo
                    )
                    resultado = "Rechazado"
                # If chain was aborted.
                else:
                    trayectoria_de_abortado = info[2][0]
                    memoria_instrucciones = info[2][2]
                    secuencia = self.procesadorCadenas.obtenerSecuenciaEstadoInstruccion(
                        memoria_instrucciones, 
                        trayectoria_de_abortado
                    )
                    resultado = "Abortado"
                    
            #print("Secuencia: ", secuencia)
                    
            # Guardar y/o imprimir información de la secuencia.
            """
            Guarda la cadena procesada y la secuencia deseada.
            """
            if imprimirPantalla:
                self.procesadorCadenas.imprimirSecuencias([secuencia], resultado)
            self.procesadorCadenas.guardarSecuencias(file, [secuencia], cadena, resultado)
                
            if imprimirPantalla: 
                for element in info[3:]:
                    print(element, end = "\t")
                print("")
                print("")
                
            file.write(str(info[3]) + "\t") # Numero de posibles procesamientos.
            file.write(str(info[4]) + "\t") # Numero de procesamiento de aceptación.
            file.write(str(info[5]) + "\t") # Número de procesamientos abortados.
            file.write(str(info[6]) + "\t") # Número de procesamientos recahazados
            file.write(str(info[7]) + "\n") # Si o no
            
        file.close()
        return "Archivo creado con éxito"
    
    def procesarCadenaConversion(self, cadena): 
        afd_equivalente = TransformadorAutomatas().AFN_LambdaToAFD(self, imprimirResultados = False)
        return afd_equivalente.procesarCadena(cadena)
        
    def procesarCadenaConDetallesConversion(self, cadena):
        afd_equivalente = TransformadorAutomatas().AFN_LambdaToAFD(self, imprimirResultados = False)
        return afd_equivalente.procesarCadenaConDetalles(cadena)
    
    def procesarListaCadenasConversion(self, listaCadenas, nombreArchivo, imprimirPantalla):
        afd_equivalente = TransformadorAutomatas().AFN_LambdaToAFD(self)
        afd_equivalente.procesarListaCadenas(listaCadenas, nombreArchivo, imprimirPantalla) 
    
    def procesar(self, cadena, current_state = 0, ultima_instruccion = "", lambdaClausura = False):

        try:
        
            # Get new lambda states.
            newLambdaStatesNumbers = self.obtenerNumerosEstadosAccesibles(
                current_state, 
                self.alphabet.simbolos.index("$")   
            )

            #print("New Lambda States: ", newLambdaStatesNumbers)


            # If chain is not empty.
            if cadena != "" or newLambdaStatesNumbers != []:

                acceptance_list = list()
                abortado = 0
                """
                Si la cadena es diferente de vacio, siga buscando transiciones
                diferentes a Lambda, siempre y cuando el interés no sea hallar
                el conjunto de lambda clausuras.
                """
                # Get next states numbers given by instruction in the chain.
                if cadena != "" and lambdaClausura == False:
                    newStatesNumbers = self.obtenerNumerosEstadosAccesibles(
                        current_state, 
                        self.alphabet.simbolos.index(cadena[0])   
                    )

                    # If there exist new states where to move.
                    if newStatesNumbers != []:
                        # Normal transitions.
                        for newState in newStatesNumbers:
                            result = self.procesar(cadena[1:], newState, cadena[0])
                            for pair in result: 
                                acceptance_list.append(pair)
                    # There is no where to move using "normal" transitions.
                    else: abortado += 1

                """
                Si existen transicion lambda disponibles, transite hacia ellas.
                """
                # Lambda transitions.
                if newLambdaStatesNumbers != []:
                    for newLambdaState in newLambdaStatesNumbers: 
                        result = self.procesar(cadena, newLambdaState, "$")
                        for pair in result: 
                            acceptance_list.append(pair)
                # There is no where to move using lambda transitions.
                else: 
                    if not lambdaClausura: abortado += 1
                    else: abortado = 2

                # Aborted! Since there is no where to move.
                if abortado == 2: return [[str(current_state), 0, ultima_instruccion]]    # Aborted.  
                else:
                    """
                    Añada la memoria del recorrido.
                    """
                    # Añadir memoria de estados e instrucciones.
                    for pair in acceptance_list: 
                        #print("Pair: ", pair)

                        # Actualizar la memoria de estados por los cuales
                        # transitó el autómata.
                        memory = pair[0] # Get trayectory.                    
                        pair[0] = str(current_state) + memory

                        # Actualizar la lista de instrucciones que ejecutó
                        # el automata.
                        instructions = pair[2]
                        pair[2] = str(ultima_instruccion) + instructions

                    return acceptance_list

            # If chain is empty.
            else:             
                if current_state in self.valid_states:
                    return [[str(current_state), 1, ultima_instruccion]]    # Accepted.

                else:
                    return [[str(current_state), -1, ultima_instruccion]]  # Rejected.
        
        except: 
            print("Existen transiciones lambda que son redudantes o inútiles, que generan que el autómata se quede en una secuencia de transiciones infinitas. Por favor, elíminelas.")

### 5. Transformaciones de Autómatas.

In [9]:
class TransformadorAutomatas: 
    def __init__(self):
        self.automata = None
        
    def setAutomata(self, automata): 
        self.automata = automata
             
    """
    Transforma AFN a AFD.
    """
    def AFNtoAFD(self, automata, imprimirResultados = True): # Falta que elimine estados innecesarios!!!
        
        # Imprimir mensaje.
        if imprimirResultados: 
            print("-----------")
            print("Conversión de autómata no-determinista a determinista.")
            print("-----------")
        
        # Determinar automata como instancia de la clase.
        self.setAutomata(automata)
        
        # Hallar nueva lista de estados y nueva función de transición.
        delta_extendida = list()
        estados = [[x] for x in list(range(len(self.automata.states)))]
        nuevos_estados = estados.copy()
        
        # Este método actualiza la lista delta_extendida y la lista de nuevos estados.
        self.obtenerEstados_y_Transiciones_AFNtoAFD(estados, delta_extendida, nuevos_estados)
        
        # Encontrar los estados válidos del autómata.
        new_valid_states = list()
        for transicion in delta_extendida: 
            for valid_state in self.automata.valid_states: 
                if valid_state in transicion[1] and transicion[1] not in new_valid_states: 
                    new_valid_states.append(transicion[1])
                    
        # Formatear valores.
        nuevos_estados, delta_extendida, new_valid_states = self.formatearValores(
            nuevos_estados, 
            delta_extendida, 
            new_valid_states
        )
        
        # Imprimir cambios.
        if imprimirResultados: 
            self.imprimirCambios(nuevos_estados, delta_extendida)

        # Crear nuevo AFD.
        alphabet = self.automata.alphabet.simbolos
        initial_state = self.obtenerNombreEstado([self.automata.initial_state])
        states = nuevos_estados.copy()
        delta = delta_extendida.copy()
        valid_states = new_valid_states.copy()
    
        # Iniciar AFD.
        newAFD = AFN(alphabet = alphabet, 
                  initial_state = initial_state, 
                  valid_states = valid_states, 
                  states = states, 
                  delta = delta, 
                 )
        
        # Certificado de ser un AFD.
        newAFD.automata_type = 0 
        return newAFD
    
    
    """
    Transforma AFN_Lambda a AFN.
    """
    def AFN_LambdaToAFN(self, automata, imprimirResultados = True):
        
        # Imprimir mensaje.
        if imprimirResultados:
            print("-----------")
            print("Conversión de autómata con transiciones Lambda a no determinista.")
            print("-----------")
        
        # Determinar automata como instancia de la clase.
        self.setAutomata(automata)
        
        # Hallar nueva función de transición.
        delta_extendida = self.obtenerEstados_y_Transiciones_AFN_LambdatoAFN(imprimirResultados)
        
        # Encontrar los estados válidos del autómata.
        new_valid_states = list()
        for state in self.automata.states: 
            clausura = self.automata.calcularLambdaClausura(state, index = True)
            valid = False
            for clausura_state in clausura: 
                for valid_state_autom in self.automata.valid_states: 
                    if clausura_state == valid_state_autom: 
                        valid = True
                        break
            if valid: 
                new_valid_states.append(state)
        
        # Imprimir cambios.
        if imprimirResultados:
            self.imprimirCambios(self.automata.states, delta_extendida)
        
        # Parámetros del AFNLambda.
        alphabet = self.automata.alphabet.simbolos
        initial_state = self.automata.states[self.automata.initial_state]
        states = self.automata.states.copy()
        delta = delta_extendida.copy()
        valid_states = new_valid_states.copy()
        
        # Iniciar AFN.
        newAFN = AFN(alphabet = alphabet, 
                  initial_state = initial_state, 
                  valid_states = valid_states, 
                  states = states, 
                  delta = delta
                  )
        return newAFN
    
    """
    Transforma AFN_Lambda a AFD.
    """
    def AFN_LambdaToAFD(self, automata, imprimirResultados = True): 
        newAFD = self.AFNtoAFD(self.AFN_LambdaToAFN(automata, imprimirResultados), imprimirResultados)
        return newAFD
    
    """
    Imprime por pantalla estados y transiciones antiguas y nuevas.
    """
    def imprimirCambios(self, nuevos_estados, delta_extendida):
        
        # Imprime por pantalla estados y transiciones antiguas.
        print("Estados antiguos:")
        print(self.automata.states)
        
        print("")
        print("Función de transición antigua:")
        for transicion in self.automata.delta: 
            print(transicion)
        print("")
        
        # Imprime por pantalla nuevos estados y nueva función de transición.
        print("Estados nuevos:")
        print(nuevos_estados)
        
        print("")
        print("Función de transición nueva:")
        for transicion in delta_extendida:
            print(transicion)
        print("")
        
    """
    Obtiene el nombre del estado. 
    Entrada: [0, 1, 2]
    Salida: ["Q0", "Q1", "Q2"]
    """
    def obtenerNombreEstado(self, estadoLista):
        estadoNombre = "{"
        for estadoIndex in estadoLista: 
            estadoNombre += self.automata.states[estadoIndex]
            if estadoLista.index(estadoIndex) != len(estadoLista) - 1: 
                estadoNombre += ","
        estadoNombre += "}"
        return estadoNombre
    
    """
    Formatea lo valores de los estados, función delta, y estados válidos de números, a su nombre original.
    """
    def formatearValores(self, nuevosEstados, nuevaFuncionDelta, new_valid_states): 

        # Formatear estados.
        estados = list()
        for estadoLista in nuevosEstados:
            estados.append(self.obtenerNombreEstado(estadoLista))
            
        # Formatear función delta.
        for transicion in nuevaFuncionDelta: 
            transicion[0] = self.obtenerNombreEstado(transicion[0])
            transicion[1] = self.obtenerNombreEstado(transicion[1])
            
        # Formatear new valid states.
        valid_states = list()
        for valid_state in new_valid_states: 
            valid_states.append(self.obtenerNombreEstado(valid_state))
        
        return estados, nuevaFuncionDelta, valid_states
        
    """
    Obtiene de manera recursiva los estados y las transiciones de un AFD generador a partir de un AFN.
    """
    def obtenerEstados_y_Transiciones_AFNtoAFD(self, estados, delta_extendida, nuevos_estados):

        for instruction in self.automata.alphabet.simbolos: 
            
            """
            estados = [[0], [1], [2]], [[0, 1], [0, 2]]
            """
            for listaEstados in estados: 
                transiciones_estado = list()
                for idx_estado in listaEstados: 
                    result = self.automata.procesar(instruction, idx_estado)            
                    for res in result: 
                        # Si el proceso no fue abortado
                        if res[1] != 0:
                            next_state = int(res[0][-1])
                            if next_state not in transiciones_estado:
                                transiciones_estado.append(next_state)
                if transiciones_estado != []:
                    delta_extendida.append([listaEstados, transiciones_estado, instruction])
                
        nuevos_estados_por_explorar = list()
        for transicion in delta_extendida: 
            if transicion[1] != [] and transicion[1] not in nuevos_estados:
                nuevos_estados.append(transicion[1])
                nuevos_estados_por_explorar.append(transicion[1])
                
        if nuevos_estados_por_explorar != []:
            self.obtenerEstados_y_Transiciones_AFNtoAFD(nuevos_estados_por_explorar, delta_extendida, nuevos_estados)
        return delta_extendida
    
    """
    Obtiene los estados y transiciones de un AFN dado un AFN_Lambda.
    Aquí no es necesario utilizar la recursión.
    """
    def obtenerEstados_y_Transiciones_AFN_LambdatoAFN(self, imprimirResultados = True):
    
        delta_extendida = list()
        for index_estado in list(range(len(self.automata.states))): 
            clausura_estado = self.automata.calcularLambdaClausura(
                current_state = self.automata.states[index_estado],
                index = True
            )
            
            # Imprimir Estado y Lambda Clausura
            nameState = self.automata.states[index_estado]
            nameClausuras = self.obtenerNombreEstado(clausura_estado)
            if imprimirResultados: 
                print("Estado: ", nameState + ". ", 
                     " Lambda Clausura: ", nameClausuras)
                print("")

            for instruction in self.automata.alphabet.simbolos: 
                if instruction != "$": 
                    
                    lista_estados = list()
                    
                    if imprimirResultados: 
                        print("Delta'(" + nameState + ", " + instruction + ") = ", end = " ")
                        print("$[Delta($[" + nameState + "]," + instruction + ")] = ", end = " ")
                        print("$[Delta(", nameClausuras, ", " + instruction + ")] = ", end = " ")
                    
                    for estado in clausura_estado:
                        result = self.automata.procesar(instruction, current_state = estado)
                        for res in result: 
                            if res[2] != "":
                                if res[2][0] == instruction: 
 
                                    numLambdaTrans = len(res[2][1:])
                                    next_state = res[0][0: len(res[0]) - numLambdaTrans][-1]
                                    lista_estados.append(int(next_state))
                    
                    if imprimirResultados: 
                        print("$[", self.obtenerNombreEstado(lista_estados), "] = ", end = " ")
                    nuevaClausura = list()
                    for estado in lista_estados:
                        clausuraEstados = self.automata.calcularLambdaClausura(
                            current_state = self.automata.states[estado],
                            index = True
                        )
                        
                        for num_estado in clausuraEstados: 
                            if num_estado not in nuevaClausura: 
                                delta_extendida.append([
                                    self.automata.states[index_estado], 
                                    self.automata.states[num_estado], 
                                    instruction
                                ])
                                nuevaClausura.append(num_estado)
                    if imprimirResultados: print(self.obtenerNombreEstado(nuevaClausura))
            if imprimirResultados: print("")
        return delta_extendida               

    """
    Calcula el complemento de un AFD.
    Entrada: AFD
    Salida: AFD Complemento.
    """
    def hallarComplemento(self, afd):
        
        # Si no es un autómata determinista, aborte el procedimiento.
        if afd.automata_type != 0: 
            return "El autómata ingresado no es un autómata determinista."
        
        # Copiar la instancia del autómata en una nueva variable.
        newAFD = copy.deepcopy(afd)
        new_validStates = list()
        
        # Modificar estados de aceptación.
        for state in range(len(newAFD.states)):
            if state not in newAFD.valid_states:
                new_validStates.append(state)
        newAFD.valid_states = new_validStates
        
        # Retornar nuevo autómata.
        return newAFD
    
    def hallarProductoCartesianoY(self, afd1, afd2, imprimirResultados = True):
        return self.__hallarProductoCartesiano(afd1, afd2, operacion = "intersección", imprimirResultados = imprimirResultados)
        
    def hallarProductoCartesianoO(self, afd1, afd2, imprimirResultados = True):
        return self.__hallarProductoCartesiano(afd1, afd2, operacion = "unión", imprimirResultados = imprimirResultados)
    
    def hallarProductoCartesianoDiferencia(self, afd1, afd2, imprimirResultados = True):
        return self.__hallarProductoCartesiano(afd1, afd2, operacion = "diferencia", imprimirResultados = imprimirResultados)

    def hallarProductoCartesianoDiferenciaSimetrica(self, afd1, afd2, imprimirResultados = True):
        return self.__hallarProductoCartesiano(afd1, afd2, operacion = "diferencia simétrica", imprimirResultados = imprimirResultados)
    
    def __hallarProductoCartesiano(self, afd1, afd2, operacion = "unión", imprimirResultados = True):
        
        # Tenga en cuenta que los alfabetos ya están ordenados desde la inicialización de los autómatas.
        if afd1.alphabet.simbolos != afd2.alphabet.simbolos: 
            return "Los autómatas ingresados no tienen el mismo alfabeto."
        
        # Define el producto cartesiano deseado.
        operaciones = {"unión": 0, "intersección": 1, "diferencia": 2, "diferencia simétrica": 3}
        operacion = operaciones[operacion]

        # Calcular nuevos estados.
        nuevos_estados = list()
        for estado1 in afd1.states: 
            for estado2 in afd2.states:
                nuevos_estados.append([estado1, estado2])
                
        """Encontrar nuevos estados de aceptación del autómata de acuerdo con la operación deseada."""
        new_valid_states = list()
        for estado in nuevos_estados: 
            
            aceptado1 = bool(estado[0] in afd1.getValidStates())
            aceptado2 = bool(estado[1] in afd2.getValidStates())
            nombreEstado = "{" + estado[0] + "," + estado[1] + "}"
            
            # Union.
            if operacion == 0:
                if aceptado1 or aceptado2: 
                    new_valid_states.append(nombreEstado)
                    
            # Intersección.
            elif operacion == 1: 
                if aceptado1 and aceptado2: 
                    new_valid_states.append(nombreEstado)
          
            # Diferencia.
            elif operacion == 2: 
                if aceptado1 and not aceptado2: 
                    new_valid_states.append(nombreEstado)
            
            # Diferencia simétrica.
            elif operacion == 3: 
                if (aceptado1 and not aceptado2) or (not aceptado1 and aceptado2): 
                    new_valid_states.append(nombreEstado)

        """ Calcular nueva función de transición."""
        delta_extendida = list()
        for estado in nuevos_estados: 
            for instruction in afd1.alphabet.simbolos: 
                
                # Buscar transición en delta del AFD1.
                for transicion in afd1.delta: 
                    if transicion[0] == estado[0] and transicion[2] == instruction: 
                        estado1 = transicion[1]
                
                # Buscar transición en delta del AFD2.
                for transicion in afd2.delta: 
                    if transicion[0] == estado[1] and transicion[2] == instruction: 
                        estado2 = transicion[1]
                
                # Imprimir transiciones.
                if imprimirResultados:
                    print("Delta(" + "(" + estado[0] + "," + estado[1] + ")" + "," + instruction + ") =", end = " ")
                    print("(Delta_1(" + estado[0] + "," + instruction + "),", end = "")
                    print("Delta_2(" + estado[1] + "," + instruction + ")) =", end = " ")
                    print("(" + estado1 + "," + estado2 + ")", end = "\n")
                
                nombreEstado = "{" + estado[0] + "," + estado[1] + "}"
                nombreNuevoEstado = "{" + estado1 + "," +  estado2 + "}"
                delta_extendida.append([nombreEstado, nombreNuevoEstado, instruction])
        
        # Parámetros del AFD. 
        alphabet = afd1.alphabet.simbolos
        initial_state = "{" + afd1.states[afd1.initial_state] + "," + afd2.states[afd2.initial_state] + "}"    
        states = ["{" + estado[0] + "," + estado[1] + "}" for estado in nuevos_estados]
        delta = delta_extendida.copy()
        valid_states = new_valid_states.copy()
        
        # Iniciar el nuevo AFD Producto Cartesiano con la operación deseada.
        newAFN = AFN(alphabet = alphabet, 
                  initial_state = initial_state, 
                  valid_states = valid_states, 
                  states = states, 
                  delta = delta
                  )
        
        # Certificado de ser un AFD.
        newAFN.automata_type = 0
        return newAFN
    
    """
    Elimina los estados innaccesibles y calculr un AFD equivalente con 
    el mínimo número de estados de acuerdo con el algoritmo estudiado en clase.
    """
    def simplificarAFD(self, afd, imprimirResultados = True): 
        
        # Set automata.
        self.setAutomata(afd)
        
        # Obtener estados inaccesibles.
        estados_inaccesibles = afd.hallarEstadosInaccesibles()
    
        # Crear nuevo AFD.
        newAFD = copy.deepcopy(afd)
        
        # Eliminar estados inaccesibles.
        new_states = list()
        for state in afd.states: 
            if state not in estados_inaccesibles: 
                new_states.append(state)
                
        # Eliminar transiciones donde hay estados inaccesibles.
        new_delta = list()
        for transicion in afd.delta: 
            if transicion[0] not in estados_inaccesibles and transicion[1] not in estados_inaccesibles:
                new_delta.append(transicion)
        
        # Eliminar estados de aceptación que no están dentro de los nuevos estados.
        new_valid_states = list()
        for state in afd.getValidStates(): 
            if state in new_states:
                new_valid_states.append(new_states.index(state))
        
        # Modificar estados y función delta.
        newAFD.states = new_states.copy()
        newAFD.delta = new_delta.copy()
        newAFD.valid_states = new_valid_states.copy()
        
        # Retorna AFD equivalente tras eliminar estados innecesarios.
        return self.algoritmoPorLlenadoTabla(newAFD, imprimirResultados)

    """
    Calcular AFD equivalente con el mínimo número de estados.
    Algoritmo por llenado de tabla para terminar equivalencia de estados en un AFD.
    """
    def algoritmoPorLlenadoTabla(self, afd, imprimirResultados):
        
        # Create triangular matrix.
        n = len(afd.states)
        triangularMatrix = np.zeros((n, n))
        
        # Todas las entradas de la diagonal son iguales a 1.
        for i in range(n):
            triangularMatrix[i][i] = 1
        
        # Actualizar matriz con estados NO equivalentes.
        tempAFD = self.hallarProductoCartesianoDiferenciaSimetrica(afd, afd, imprimirResultados = False)
        estadosNoEquivalentesNombre = tempAFD.getValidStates()
        
        estadosNoEquivalentesIndex = list()
        for estado in estadosNoEquivalentesNombre:
            indexEstado = self.obtenerIndexEstado(estado)
            estadosNoEquivalentesIndex.append(indexEstado)
            triangularMatrix[indexEstado[0]][indexEstado[1]] = 1
        
        # Por cada posición de la matriz que no tenga 1 (no ha sido marcada), calcule transiciones.
        # Repita hasta que no hayan más pares de estados marcados.
        num_estados_eliminados = 1
        iteracion = 1
        while num_estados_eliminados > 0:
            iteracion += 1
            num_estados_eliminados = 0
            for i in range(n):
                for j in range(n):
                    if triangularMatrix[i][j] == 0: 
                        for transicion in tempAFD.delta:
                            if self.obtenerIndexEstado(transicion[0]) == [i, j]: 
                                if self.obtenerIndexEstado(transicion[1]) in estadosNoEquivalentesIndex: 
                                    if [i, j] not in estadosNoEquivalentesIndex:
                                        num_estados_eliminados += 1
                                        estadosNoEquivalentesIndex.append([i, j])
                                        triangularMatrix[i][j] = iteracion
                                        triangularMatrix[j][i] = iteracion
    
        # Calcular nuevos estados del autómata.
        index_estados_ya_incluidos = list()
        nuevos_estados_index = list()
        
        for i in range(n):
            if i not in index_estados_ya_incluidos:
                index_estados_ya_incluidos.append(i)
                pos_equivalentes = [i]
                
                for j in range(i, n): 
                    if triangularMatrix[i][j] == 0:
                        if j not in index_estados_ya_incluidos:
                            pos_equivalentes.append(j)
                            index_estados_ya_incluidos.append(j)
                nuevos_estados_index.append(pos_equivalentes)
                index_estados_ya_incluidos.sort()  
            if index_estados_ya_incluidos == list(range(len(afd.states))):
                break
                
        # Calcular nueva función de transición del autómata.
        delta_extendida = list()
        
        # Por cada estado en la lista de nuevos estados.
        for estado_index in nuevos_estados_index:
            
            # Por cada símbolo en el alfabeto.
            for instruction in afd.alphabet.simbolos:
                
                # Inicializar y computar lista de index de los estados a los que se pueden transitar.
                lista_index_transicion = list()
                for idx in estado_index:
                    res = afd.procesar(instruction, current_state = idx)
                    index_nuevoEstado = int(res[0][-1])
                    lista_index_transicion.append(index_nuevoEstado)
                    
                # Buscar en qué estado está incluido lista_index_transicion
                nuevoEstado = self.buscarEstadoInclusion(lista_index_transicion, nuevos_estados_index)

                # Agregar a la función delta la nueva transición.
                nombreEstado = self.obtenerNombreEstado(estado_index)
                nombreNuevoEstado = self.obtenerNombreEstado(nuevoEstado)
                delta_extendida.append([nombreEstado, nombreNuevoEstado, instruction])
                
        # Calcular nombre de los nuevos estados, estado inicial y estados de aceptación.
        new_valid_states = list()
        nuevos_estados_nombre = list()
        for estado_index in nuevos_estados_index: 
            
            # Calcular nombre de los estados.
            nuevos_estados_nombre.append(self.obtenerNombreEstado(estado_index))
            
            # Encontrar estado inicial.
            if afd.initial_state in estado_index: 
                new_initial_state = self.obtenerNombreEstado(estado_index)
            
            # Encontrar estados de aceptación del autómata.
            for estado_aceptacion_index in afd.valid_states: 
                if estado_aceptacion_index in estado_index: 
                    if self.obtenerNombreEstado(estado_index) not in new_valid_states:
                        new_valid_states.append(self.obtenerNombreEstado(estado_index))
       
        # Imprimir resultados.
        if imprimirResultados: 
            print("Nuevos estados del autómata: ")
            print(nuevos_estados_nombre)
            print("")

            print("Nueva transición delta:")
            for transicion in delta_extendida:
                print(transicion)
            print("")

            print("Nuevo estado inicial:")
            print(new_initial_state)
            print("")

            print("Nuevos estados de aceptación:")
            print(new_valid_states)
            print("")

            # Imprimir matriz triangular.
            print("Resultado algorítmo por llenado de tabla.")
            count = 1
            for i in range(1, n):
                copia = list(triangularMatrix[i][0: count].copy())
                for element in copia: 
                    if element == 0: 
                        print("E" , end = " ")
                    else: 
                        print(int(element), end = " ")
                print("")
                count += 1

        # Parámetros del AFD. 
        alphabet = afd.alphabet.simbolos
        initial_state = new_initial_state
        states = nuevos_estados_nombre.copy()
        delta = delta_extendida.copy()
        valid_states = new_valid_states.copy()
        
        # Iniciar el nuevo AFD.
        newAFD = AFN(alphabet = alphabet, 
                  initial_state = initial_state, 
                  valid_states = valid_states, 
                  states = states, 
                  delta = delta
                  )
        
        # Certificado de ser un AFD.
        newAFD.automata_type = 0
        
        # Retornar instancia de AFD.
        return newAFD

        
    """
    Busca el estado en el que están incluídos los elementos de estado_lista.
    Entrada: estado_lista = [0, 1, 2], estados = [[0, 1, 2, 3], [6, 3, 7, 8]]
    Salida: [0, 1, 2, 3]
    """
    def buscarEstadoInclusion(self, estado_lista, estados_candidatos):
        for candidato in estados_candidatos:
            inclusion = True
            for estado_index in estado_lista: 
                if estado_index not in candidato: 
                    inclusion = False
                    break
            if inclusion: 
                return candidato
                
    """
    Entrada: Nombre Estado
    Salida: Index.
    
    Ejemplo: 
    Entrada: '{Q0,Q1}'
    Salida: [0, 1]
    """
    def obtenerIndexEstado(self, estado):
        pos = 0
        instances_index = list()
        instance_pos = estado[pos:].find("},{")
        while instance_pos != -1:
            instances_index.append(instance_pos)
            pos = instance_pos + len("},{")
            if estado[pos:].find("},{") != -1:
                instance_pos = pos + estado[pos:].find("},{")
            else: 
                break
        if len(instances_index) == 0:
            indexState = estado[1: -1].split(",")
        else: 
            if len(instances_index) == 1:
                format_pos = instances_index[0]
            else: 
                format_pos = int((len(instances_index)-1)/2)
                format_pos = instances_index[format_pos]
            indexState = estado[: format_pos + 1] + " " + estado[format_pos + 2:]
            indexState = indexState.split(" ")
        for i in range(len(indexState)):
            indexState[i] = self.automata.states.index(indexState[i])
        return indexState

### 6. Clase Validación.

In [10]:
class ClaseValidacion: 
    
    def __init__(self):
        self.transformador = TransformadorAutomatas()
    
    def generarCadenasAleatorias(self, automata):
        lista_cadenas = list()
        for i in range(5000):
            longitud_cadena = np.random.randint(200)
            cadena = automata.alphabet.generarCadenaAleatoria(longitud_cadena)
            lista_cadenas.append(cadena)
        return lista_cadenas
    
    def validarAFNtoAFD(self, listaAFN): 
        
        for afn in listaAFN: 
            
            print("---Autómata AFN No." + str(listaAFN.index(afn) + 1) + "---")
            
            # Inicializar variables.
            coincide = 0
            no_coincide = 0
            cadenas_error = list()

            # Generar 5000 cadenas aleatorias.
            cadenas = self.generarCadenasAleatorias(afn)

            for cadena in cadenas: 
                if afn.procesarCadena(cadena) == afn.procesarCadenaConversion(cadena): 
                    coincide += 1
                else: 
                    no_coincide += 0
                    cadenas_error.append(cadena)

            print("* Número de casos en el que se obtuvo: *")
            print("1. Mismo resultado: ", coincide)
            print("2. Resultado diferente: ", no_coincide)
            print("")

            print("* Cadenas que dieron diferentes resultados: *")
            for error in cadenas_error:
                print(error)
            print("")
    
    def validarAFNLambdaToAFN(self, listaAFNLambda): 

        for afnLambda in listaAFNLambda: 
            
            print("Autómata AFNLambda No." + str(listaAFNLambda.index(afnLambda) + 1) + "---")

            # Calcular AFN Equivalente.
            afn = self.transformador.AFN_LambdaToAFN(afnLambda, imprimirResultados = False)

            # Inicializar variables.
            coincide = 0
            no_coincide = 0
            cadenas_error = list()

            # Generar 5000 cadenas aleatorias.
            cadenas = self.generarCadenasAleatorias(afnLambda)

            for cadena in cadenas: 
                if afnLambda.procesarCadena(cadena) == afn.procesarCadena(cadena): 
                    coincide += 1
                else: 
                    no_coincide += 1
                    cadenas_error.append(cadena)

            print("-- Número de casos en el que se obtuvo: --")
            print("1. Mismo resultado: ", coincide)
            print("2. Resultado diferente: ", no_coincide)
            print("")

            print("-- Cadenas que dieron diferentes resultados: --")
            for error in cadenas_error:
                print(error)
            print("")

### 7. Clase MenuOpciones.

In [11]:
class MenuOpciones: 
    
    def __init__(self, titulo, opciones_texto, lista_opciones_validas, exit = True):
        self.titulo = titulo
        self.opciones_texto = opciones_texto
        self.lista_opciones_validas = lista_opciones_validas
        self.lista_opciones_validas.append(self.lista_opciones_validas[-1] + 1)
        self.exit = exit
        
    def run(self):
        print("---" + self.titulo + "---")
        for opcion_idx in range(len(self.opciones_texto)): 
            print(str(opcion_idx + 1) + "." + self.opciones_texto[opcion_idx])
        if self.exit:
            print("0:Finalizar Programa.")
        opcion = self.validarOpcion(self.lista_opciones_validas) 
        return opcion
        
    def validarOpcion(self, lista_opciones):
        try:
            opcionValidar = int(input("Digite una opción: "))
            while(int(opcionValidar) not in lista_opciones):
                opcionValidar = int(input("Opción inválida, digite una de las opciones señaladas en el menú: "))
        except: 
            print("Error. La opción digitada no es un número.")
            opcionValidar = self.validarOpcion(lista_opciones)
        return opcionValidar

### 8. ClasePrueba.

Esta clase permite al usuario interactuar con una interfaz gráfica de AUTOM. Entre algunas de sus funciones se encuentran probar inicializar y ejecutar funciones con autómatas deterministas, no deterministas y no deterministas con transiciones lambda. Adicionalmente, permite cargar autómatas mediante archivo.

In [12]:
class ClasePrueba:
    
    """
    Invoca a los otros para que puedan ser comentados fácilmente y poder 
    escoger cuál se va a probar.
    """
    def __init__(self):
        self.transformador = TransformadorAutomatas()
        self.automata = None

        
    def inicializarAutomata(self, tipo_opcion = 1):
        print("")
        print("Importante: El archivo debe tener el formato indicado en el manual de usuario de AutomLib.")
        filename = input("Digite ruta del archivo: ")
        print("")
        automata_ = None
        try:
            #automata_type = ProcesadorArchivos(filename).automata_type
            if tipo_opcion == 1:
                automata_ = AFD(filename = filename)
            elif tipo_opcion == 2:
                automata_ = AFN(filename = filename)
            elif tipo_opcion == 3:
                automata_ = AFNLambda(filename = filename)  
            return automata_
        except: 
            print("")
            print("Ha ocurrido un error al intentar inicializar el autómata.")
            print("Posibles razones: ")
            print("1.Está intentando inicializar un autómata determinista, y no todas las transiciones están definidas.")
            print("2.El archivo no satisface el formato establecido en el manuel de usuario de AutomLib")
            print("")
        return automata_
    
    
    def main(self):
        
        # Menu principal.
        opcionAutom = MenuOpciones(
            titulo = "Bienvenid@ a la Librería Autom", 
            opciones_texto = [
                "Probar Autómata Determinista",
                "Probar Autómata No-Determinista",
                "Probar Autómata No-Determinista con Transiciones Lambda",
            ], 
            lista_opciones_validas = list(range(3))
        ).run()
        if opcionAutom == 0:
            print("Programa finalizado con éxito.")
            return
                
        # Inicializar autómata con archivo.
        print("Para inicializar el autómata, digite la dirección del archivo.")
        self.automata = self.inicializarAutomata(opcionAutom)
        
        # Si no se pudo inicializar el autómata, finalice la aplicación.
        if self.automata == None: 
            print("Programa finalizado.")
            return 
        
        # Ejecutar función adecuada de acuerdo con la opción escogida.
        functionName = {1: "probarAFD", 2: "probarAFN", 3: "probarAFNLambda"}
        print("Opción Digitada: ", functionName[opcionAutom])
        print("")
        getattr(self, functionName[opcionAutom])()

    """
    --- SubMenús ---
    """
        
    # Probar Autómata Determinista.
    def probarAFD(self):
        opcionAutom = MenuOpciones(
            titulo = "Menú Autómata Determinista", 
            opciones_texto = [
                "Procesar cadena",
                "Procesar cadena con detalles",
                "Procesar lista de cadenas",
                "Calcular complemento", 
                "Calcular producto cartesiano",
                "Simplificar autómata",
                "Regresar al menú principal"
            ], 
            lista_opciones_validas = list(range(6))
        ).run()
        if opcionAutom == 0:
            print("Programa finalizado con éxito.")
            return
        elif(opcionAutom == 7):
            print("")
            self.main()
            return
        
        functionName = {
                        1: "opcion_procesarCadena", 
                        2: "opcion_procesarConDetalles", 
                        3: "opcion_procesarListaCadenas", 
                        4: "probarComplemento", 
                        5: "probarProductoCartesiano",
                        6: "probarSimplificacion"
                        }
        print("Opción Digitada: ", functionName[opcionAutom])
        print("")
        getattr(self, functionName[opcionAutom])()
        
    # Probar Autómata No-Determinista.
    def probarAFN(self):
        opcionAutom = MenuOpciones(
            titulo = "Menú Autómata No-Determinista", 
            opciones_texto = [
                "Procesar cadena",
                "Procesar cadena con detalles",
                "Computar todos los procesamientos",
                "Procesar lista de cadenas",
                "Probar AFN a AFD",
                "Regresar al menú principal",
            ], 
            lista_opciones_validas = list(range(7))
        ).run()
        if opcionAutom == 0:
            print("Programa finalizado con éxito.")
            return
        elif(opcionAutom == 6):
            print("")
            self.main()
            return
        functionName = {
                        1: "opcion_procesarCadena", 
                        2: "opcion_procesarConDetalles", 
                        3: "opcion_computarTodosLosProcesamientos", 
                        4: "opcion_procesarListaCadenas", 
                        5: "probarAFNtoAFD"
                       }
        print("Opción Digitada: ", functionName[opcionAutom])
        print("")
        getattr(self, functionName[opcionAutom])()
        
    
    # Probar Autómata No-Determinista Lambda.
    def probarAFNLambda(self):
        
        opcionAutom = MenuOpciones(
            titulo = "Menú Autómata Lambda", 
            opciones_texto = [
                "Procesar cadena",
                "Procesar cadena con detalles",
                "Computar todos los procesamientos",
                "Procesar lista de cadenas",
                "Calcular Lambda Clausura de un estado",
                "Calcular Lambda Clausura de un conjunto de estados",
                "Probar AFNLambda a AFN",
                "Probar AFNLambda a AFD",
                "Regresar al menú principal",
            ], 
            lista_opciones_validas = list(range(9))
        ).run()
        if opcionAutom == 0:
            print("Programa finalizado con éxito.")
            return
        elif(opcionAutom == 9):
            print("")
            self.main()
            return
        functionName = {
                        1: "opcion_procesarCadena", 
                        2: "opcion_procesarConDetalles", 
                        3: "opcion_computarTodosLosProcesamientos", 
                        4: "opcion_procesarListaCadenas",
                        5: "opcion_calcularLambdaClausura", 
                        6: "opcion_calcularLambdaClausuras",
                        7: "probarAFNLambdaToAFN", 
                        8: "probarAFNLambdaToAFD"
                       }        
        print("Opción Digitada: ", functionName[opcionAutom])
        print("")
        getattr(self, functionName[opcionAutom])()
        


    """
    ***** Funciones de Opciones ****
    """

    # Regresar a submenú.
    def regresarASubMenu(self):
        print("")
        menuName = {0: "probarAFD", 1: "probarAFN", 2: "probarAFNLambda"}
        getattr(self, menuName[self.automata.automata_type])()
    
    def preguntarRepetirOpcion(self):
        print("")
        opcionAutom = MenuOpciones(
            titulo = "¿Desea volver a probar la función?", 
            opciones_texto = [
                "Sí",
                "No",
            ], 
            lista_opciones_validas = list(range(1, 2)),
            exit = False
        ).run()
        print("")
        if opcionAutom == 1: return True
        else: return False

    # Probar AFN to AFD.
    def probarAFNtoAFD(self): 
        
        repetir = True
        while repetir: 
            print("Digite una cadena: ")
            print("Ejemplo: abaaaba")
            print("")
            cadena = input("Digite cadena: ")
            
            #try:
            # ProcesarCadena en autómata 1.
            print("AFN acepta: " + cadena + " => ", end = " ") 
            res1 = self.automata.procesarCadena(cadena)
            print(res1)

            # ProcesarCadena en autómata 2.
            automata2 = self.transformador.AFNtoAFD(self.automata, imprimirResultados = False)
            print("AFD acepta cadena: ", end= " ")
            res2 = automata2.procesarCadena(cadena)
            print(res2)

            # Compare si en ambos casos cada cadena es aceptada o rechazada.
            print("Coinciden: ", res1 == res2)

            # Imprime automatas.
            print("")
            print("**Autómata No-Determinista**")
            self.automata.toString()
            print("")

            print("")
            print("**Autómata Determinista**")
            automata2.toString()
            print("")

            #except: 
            #    print("La cadena ingresada no es válida.")
            repetir = self.preguntarRepetirOpcion()
    
        # Regresar a submenú.
        self.regresarASubMenu()
    
    
    # Probar AFNLambda to AFN.
    def probarAFNLambdaToAFN(self):
        
        
        repetir = True
        while repetir: 
            print("Digite una cadena: ")
            print("Ejemplo: abaaaba")
            print("")
            cadena = input("Digite cadena: ")



            # ProcesarCadena en autómata 1.
            print("AFN Lambda acepta: " + cadena + " => ", end = " ") 
            res1 = self.automata.procesarCadena(cadena)
            print(res1)

            # ProcesarCadena en autómata 2.
            automata2 = self.transformador.AFN_LambdaToAFN(self.automata, imprimirResultados = False)
            print("AFN acepta cadena: ", end= " ")
            res2 = automata2.procesarCadena(cadena)
            print(res2)

            # Compare si en ambos casos cada cadena es aceptada o rechazada.
            print("Coinciden: ", res1 == res2)

            # Imprime automatas.
            print("")
            print("**Autómata Lambda**")
            self.automata.toString()
            print("")
    
            print("")
            print("**Autómata No-Determinista**")
            automata2.toString()
            print("")
            repetir = self.preguntarRepetirOpcion()
    
        # Regresar a submenú.
        self.regresarASubMenu()
        
    # Probar AFNLambda to AFD.
    def probarAFNLambdaToAFD(self):
        
        repetir = True
        while repetir: 
            print("Digite una cadena: ")
            print("Ejemplo: abaaaba")
            print("")
            cadena = input("Digite cadena: ")
            
            
            #try:
            # ProcesarCadena en autómata 1.
            print("AFN Lambda acepta: " + cadena + " => ", end = " ") 
            res1 = self.automata.procesarCadena(cadena)
            print(res1)

            # ProcesarCadena en autómata 2.
            automata2 = self.transformador.AFN_LambdaToAFN(self.automata, imprimirResultados = False)
            print("AFN acepta cadena: ", end= " ")
            res2 = automata2.procesarCadena(cadena)
            print(res2)

            # ProcesarCadena en autómata 3.
            automata3 = self.transformador.AFNtoAFD(automata2, imprimirResultados = False)
            print("AFD acepta cadena: ", end= " ")
            res3 = automata3.procesarCadena(cadena)
            print(res3)

            # Compare si en ambos casos cada cadena es aceptada o rechazada.
            print("Coinciden: ", res1 == res2 == res3)

            # Imprime automatas.
            print("")
            print("**Autómata Lambda**")
            self.automata.toString()
            print("")
                        
            print("")
            print("**Autómata No-Determinista**")
            automata2.toString()
            print("")

            print("")
            print("**Autómata Determinista**")
            automata3.toString()
            print("")
            #except: 
            #    print("La cadena ingresada no es válida.")
            repetir = self.preguntarRepetirOpcion()
            
        # Regresar a submenú.
        self.regresarASubMenu()
        
    # Calcular complemento de autómata.
    def probarComplemento(self):
        
        repetir = True
        while repetir:

            # Imprimir primer autómata.
            print("**Autómata Original**")
            print("")
            self.automata.toString()

            # Imprimir segundo autómata.
            print("**Autómata Complemento**")
            print("")
            automata2 = self.transformador.hallarComplemento(self.automata)
            automata2.toString()
            
            repetir = self.preguntarRepetirOpcion()

        # Regresar a submenú.
        self.regresarASubMenu()
        
    # Calcular producto cartesiano.
    def probarProductoCartesiano(self):
        
        repetir = True
        
        while repetir: 
            print("Inicialice el autómata con el que desea calcular el producto cartesiano: ")
            automata2 = self.inicializarAutomata()
            if automata2 == None: 
                self.regresarASubMenu()
                return

            opcionAutom = MenuOpciones(
                titulo = "Complemento deseado", 
                opciones_texto = [
                    "Y",
                    "O",
                    "Diferencia",
                    "Diferencia Simétrica"
                ], 
                lista_opciones_validas = list(range(4))
            ).run()
            if opcionAutom == 0:
                print("Programa finalizado con éxito.")
                return

            elif opcionAutom == 1: 
                automata3 = self.transformador.hallarProductoCartesianoY(self.automata, automata2, imprimirResultados = False)
            elif opcionAutom == 2: 
                automata3 = self.transformador.hallarProductoCartesianoO(self.automata, automata2, imprimirResultados = False)
            elif opcionAutom == 3: 
                automata3 = self.transformador.hallarProductoCartesianoDiferencia(self.automata, automata2, imprimirResultados = False)
            elif opcionAutom == 4: 
                automata3 = self.transformador.hallarProductoCartesianoDiferenciaSimetrica(self.automata, automata2, imprimirResultados = False)

            # Imprimir autómata producto cartesiano.
            automata3.toString()
            
            repetir = self.preguntarRepetirOpcion()
        
        # Regresar a submenú.
        self.regresarASubMenu()
    
    """
    Crear AFD, simplificarlos y revisar con algunas cadenas si aceptan o rechazan cada cadena 
    de prueba.
    """
    def probarSimplificacion(self): 
        
        repetir = True
        while repetir: 
            
            # Simplificar autómata.
            newAFD = self.transformador.simplificarAFD(self.automata, imprimirResultados = False)
            
            # Imprimir autómata producto cartesiano.
            print("**Autómata Equivalente**")
            newAFD.toString()
            
            # ¿Repetir opción?
            repetir = self.preguntarRepetirOpcion()
        
        # Regresar a submenú.
        self.regresarASubMenu()
    
    # Opción procesar cadena.
    def opcion_procesarCadena(self):
        
        repetir = True
        while repetir: 
            print("Digite una nueva cadena: ")
            print("Ejemplo: abaaaba")
            print("")
            cadena = input("")
            try:
                print(self.automata.procesarCadena(cadena))
            except: 
                print("La cadena ingresada no es válida.")
            repetir = self.preguntarRepetirOpcion()
        
        # Regresar a submenú.
        self.regresarASubMenu()
    
    # Opción procesar cadena con detalles.
    def opcion_procesarConDetalles(self):
        
        repetir = True
        while repetir: 
            print("Digite una nueva cadena: ")
            print("Ejemplo: abaabb")
            print("")
            cadena = input("")

        
            print(self.automata.procesarCadenaConDetalles(cadena))
  
                #print("La cadena ingresada no es válida.")
            repetir = self.preguntarRepetirOpcion()
            
        # Regresar a submenú.
        self.regresarASubMenu()
        
    # Opción computar todos los procesamientos.
    def opcion_computarTodosLosProcesamientos(self):
        
        repetir = True
        while repetir: 
            print("Digite una cadena.")
            print("Ejemplo: abaabb")
            print("")
            cadena = input("")
            try:
                print(self.automata.computarTodosLosProcesamientos(cadena))
            except: 
                print("La cadena ingresada no es válida: algunos caracteres no hacen parte del alfabeto.")
            repetir = self.preguntarRepetirOpcion()
            
        # Regresar a submenú.
        self.regresarASubMenu()       
    
    # Opción procesar lista de cadenas.
    def opcion_procesarListaCadenas(self):
        
        repetir = True
        while repetir: 
            print("Digite una lista de cadenas separadas por comas y sin espacios.")
            print("Ejemplo: abbbaa,bbaaab,abababa")
            print("")
            listaCadenas = input("").split(",")

            print("Digite la dirección del archivo en el cual desea guardar los resultados: ")
            print("Ejemplo: ./resultados.txt")
            print("")
            nombreArchivo = input("Respuesta: ")

            print("¿Desea imprimir los resultados en pantalla?")
            print("1.Sí")
            print("2.No")
            print("")
            imprimirPantalla = (int(input("Respuesta: ")) == 1)

            try: 
                self.automata.procesarListaCadenas(listaCadenas, nombreArchivo, imprimirPantalla)
            except: 
                print("Se ha producido un error con los valores digitados, vuelva a intentarlo.")
            repetir = self.preguntarRepetirOpcion()
        
        # Regresar a submenú.
        self.regresarASubMenu()
    
    # Opción calcular lambda clasura.
    def opcion_calcularLambdaClausura(self):
        repetir = True
        while repetir:
            print("Digite un estado del autómata: ")
            print("Ejemplo: Q0")
            print("")
            estado = input("")
            try:
                print(self.automata.calcularLambdaClausura(current_state = estado))
            except: 
                print("El estado digitado no pertenece al autómata.")
            repetir = self.preguntarRepetirOpcion()
            
        # Regresar a submenú.
        self.regresarASubMenu()
    
    # Opción calcular Lambda Clausuras.
    def opcion_calcularLambdaClausuras(self):
        repetir = True
        while repetir: 
            print("Digite una lista de estados separamos por comas y sin espacios.")
            print("Ejemplo: Q0,Q1,Q2,Q3,Q4")
            print("")
            estados = input("").split(",")
            try: 
                print(self.automata.calcularLambdaClausuras(estados))
            except: 
                print("Se ha producido un error con los valores ingresados. Vuelva a intentarlo.")
            repetir = self.preguntarRepetirOpcion()
            
        # Regresar a submenú.
        self.regresarASubMenu()